In [2]:
from utils import CovidMongo
import pandas as pd
import os
import numpy as np
import json
import re
import os

# --------- Pandas Settings ---------- #
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', -1)

/home/ubuntu/.local/share/virtualenvs/covid19-dash-q8XJLdOW/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  if sys.path[0] == '':


## Load Data

In [3]:
DATA_DIR = "../csse_covid_19_data/csse_covid_19_daily_reports/"
CSV_FILE = "03-15-2020.csv"

df = pd.read_csv(os.path.join(DATA_DIR, CSV_FILE))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Province/State  119 non-null    object 
 1   Country/Region  258 non-null    object 
 2   Last Update     258 non-null    object 
 3   Confirmed       258 non-null    int64  
 4   Deaths          258 non-null    int64  
 5   Recovered       258 non-null    int64  
 6   Latitude        258 non-null    float64
 7   Longitude       258 non-null    float64
dtypes: float64(2), int64(3), object(3)
memory usage: 16.2+ KB


In [4]:
df.head()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude
0,Hubei,China,2020-03-15T18:20:18,67794,3085,54288,30.9756,112.2707
1,NaN,Italy,2020-03-14T20:13:16,24747,1809,2335,41.8719,12.5674
2,NaN,Iran,2020-03-15T18:20:18,13938,724,4590,32.4279,53.6880
3,NaN,"Korea, South",2020-03-15T18:20:18,8162,75,510,35.9078,127.7669
4,NaN,Spain,2020-03-15T18:20:18,7798,289,517,40.4637,-3.7492


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Province/State  119 non-null    object 
 1   Country/Region  258 non-null    object 
 2   Last Update     258 non-null    object 
 3   Confirmed       258 non-null    int64  
 4   Deaths          258 non-null    int64  
 5   Recovered       258 non-null    int64  
 6   Latitude        258 non-null    float64
 7   Longitude       258 non-null    float64
dtypes: float64(2), int64(3), object(3)
memory usage: 16.2+ KB


# Wrangle

In [9]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands, U.S.': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}
def get_states(txt):
    txt = txt.strip()
    return us_state_abbrev[txt]

def wrangle(df):
    
    # Extract US
    df = df[df['Country/Region'] == 'US']
    # Remove Cruise Ships
    df = df[~ (df["Province/State"].str.endswith("Princess"))]
    # Create State col
    df["State"] = df["Province/State"].apply(get_states)
    # Re-order columns
    df = df[['Province/State', 'State', 'Country/Region', 'Latitude', 'Longitude', 'Confirmed',
       'Deaths', 'Recovered', 'Last Update']]
    # Parse datetime
    df["Last Update"] = pd.to_datetime(df["Last Update"], infer_datetime_format=True)   
    return df

df_usa = wrangle(df)
df_usa.head()

,Province/State,State,Country/Region,Latitude,Longitude,Confirmed,Deaths,Recovered,Last Update
23,New York,NY,US,42.1657,-74.9481,732,3,0,2020-03-15 18:20:19
25,Washington,WA,US,47.4009,-121.4905,643,40,1,2020-03-15 02:13:21
32,California,CA,US,36.1162,-119.6816,426,6,6,2020-03-15 18:33:03
54,Massachusetts,MA,US,42.2302,-71.5301,164,0,1,2020-03-14 22:13:19
65,Florida,FL,US,27.7663,-81.6868,115,4,0,2020-03-15 18:20:19


In [10]:
df_usa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54 entries, 23 to 256
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Province/State  54 non-null     object        
 1   State           54 non-null     object        
 2   Country/Region  54 non-null     object        
 3   Latitude        54 non-null     float64       
 4   Longitude       54 non-null     float64       
 5   Confirmed       54 non-null     int64         
 6   Deaths          54 non-null     int64         
 7   Recovered       54 non-null     int64         
 8   Last Update     54 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 4.2+ KB


### Find NAs

In [11]:
df_usa.isna().sum()

Province/State    0
State             0
Country/Region    0
Latitude          0
Longitude         0
Confirmed         0
Deaths            0
Recovered         0
Last Update       0
dtype: int64

## Load to Database

### Convert `df` to `json`

In [12]:
DATA_DIR = "../csse_covid_19_data"
full_path = os.path.join(DATA_DIR, 'state_data.json')
df_usa.to_json(full_path, orient='records')

### Dump `json` to MongoDB

In [16]:
DB_NAME = 'covid'
COLLECTION_NAME = 'state'
JSON_FILE_NAME = 'state_data.json'
easy_mongo = CovidMongo(DB_NAME, COLLECTION_NAME, verbose=True)
easy_mongo.dump_json_data_to_collection(os.path.join(DATA_DIR, JSON_FILE_NAME), verbose=True)

-------- MongoDB Atlas --------
Version: 4.2.3
Databases: 
['covid19-db', 'admin', 'local']
Collections in database covid:
[]
-------- MongoDB Data Dump Result --------
file : ../csse_covid_19_data/state_data.json
Total records inserted: 54
